In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import matplotlib.ticker as ticker
from scipy.stats import norm
%matplotlib inline

### Task 1, 2

In [2]:
def OU(X, kappa, alpha, sigma, dW, dt):
    dX = kappa * (alpha - X) * dt + sigma * dW
    return dX

In [59]:
def beta_1(X, n):
    first_term = sum(X[i] * X[i - 1] for i in range(1, n)) / n
    second_term = (sum(X) - X[0]) * (sum(X) - X[-1]) / (n ** 2)
    third_term = (sum(X ** 2) - (X[-1] ** 2)) / n
    fouth_term = ((sum(X) - X[-1]) ** 2) / (n ** 2)
    b_1 = (first_term - second_term) / (third_term - fouth_term)
    return b_1

In [63]:
beta_1(X_ou, len(X_ou))

0.9996959388005175

In [4]:
def beta_2(X, n, b_1):
    b_2 = sum(X[i] - b_1 * X[i - 1] for i in range(1, n)) / (n * (1 - b_1))
    return b_2

In [5]:
def beta_3(X, n, b_1, b_2):
    b_3 = sum((X[i] - b_1 * X[i - 1] - b_2 * (1 - b_1)) ** 2 for i in range(1, n)) / n
    return b_3

In [45]:
def theta_2_n(X, delta, n):
    first_part = sum(X[i] * X[i - 1] for i in range(1, n))
    second_part = sum(X[i - 1] ** 2 for i in range(1, n))
    t_2_n = -np.log(first_part / second_part) / delta
    return t_2_n

In [46]:
def theta_3_n(X, delta, n, t_2_n):
    first_part = sum((X[i] - X[i - 1] * np.exp(-delta * t_2_n)) ** 2 for i in range(1, n))
    second_part = (2 * t_2_n) / (n * (1 - np.exp(-2 * delta * t_2_n)))
    return first_part * second_part

In [61]:
kappa = 3
alpha = 100
sigma = 2
N = 10000
delta = 1 / N
dW = np.random.normal(0, np.sqrt(1 / N), N)
M = 200
theta_new = np.zeros((M, 3))

for j in tqdm(range(M)):
    X_ou = np.zeros(N)
    X_ou[0] = 10
    for i in range(1, N):
        dX = OU(X_ou[i - 1], kappa, alpha, sigma, dW[i], 1 / N)
        X_ou[i] = X_ou[i - 1] + dX

    b_1 = beta_1(X_ou, len(X_ou))
    b_2 = beta_2(X_ou, len(X_ou), b_1)
    b_3 = beta_3(X_ou, len(X_ou), b_1, b_2)

    kappa_new = (-1 / delta) * np.log(b_1)
    alpha_new = b_2
    sigma_new = np.sqrt(2 * kappa_new * b_3 / (1 - b_1 ** 2))
    theta_new[j] = kappa_new, alpha_new, sigma_new

100%|██████████| 200/200 [00:06<00:00, 30.67it/s]


In [62]:
print(np.mean(theta_new, axis=0))
print(theta_2_n(X_ou ,delta, len(X_ou)))
print(np.sqrt(theta_3_n(X_ou ,delta, len(X_ou), theta_2_n(X_ou ,delta, len(X_ou)))))

[ 3.04107435 99.36489002  2.02956855]
-0.7797966984794227
2.2328422166451762


In [52]:
kappa = 3
alpha = 1
sigma = 2
N = 10000
delta = 1 / N
dW = np.random.normal(0, np.sqrt(1 / N), N)
M = 200
theta_new = np.zeros((M, 3))

for j in tqdm(range(M)):
    X_ou = np.zeros(N)
    X_ou[0] = 10
    for i in range(1, N):
        dX = OU(X_ou[i - 1], kappa, alpha, sigma, dW[i], 1 / N)
        X_ou[i] = X_ou[i - 1] + dX

    b_1 = beta_1(X_ou, len(X_ou))
    b_2 = beta_2(X_ou, len(X_ou), b_1)
    b_3 = beta_3(X_ou, len(X_ou), b_1, b_2)

    kappa_new = (-1 / delta) * np.log(b_1)
    alpha_new = b_2
    sigma_new = np.sqrt(2 * kappa_new * b_3 / (1 - b_1 ** 2))
    theta_new[j] = kappa_new, alpha_new, sigma_new

100%|██████████| 200/200 [00:06<00:00, 29.26it/s]


In [53]:
print(np.mean(theta_new, axis=0))

[3.66522017 1.68724395 2.00342413]


In [8]:
# X_{t - 1}
def func_mu(X_t, kappa, alpha, sigma, delta, n):
    expected_value = X_t * np.exp(-kappa * delta) + alpha * (1 - np.exp(-kappa * delta))
    return expected_value

In [9]:
def func_vi(X_t, kappa, alpha, sigma, delta, n):
    variance = 0.5 * (sigma) * (1 - np.exp(-2 * kappa * delta)) / kappa
    return variance

In [10]:
def likelihood_function(X, X_0, kappa, alpha, sigma, delta, n):
    first_multiplier = norm(0, 1).cdf(np.sqrt(2 * kappa) * (X_0 - alpha) / np.sqrt(sigma))
    second_multiplier = 1
    for i in range(1, n):
        mu = func_mu(X[i], kappa, alpha, sigma, delta, n)
        vi = func_vi(X[i - 1], kappa, alpha, sigma, delta, n)
        second_multiplier *= np.prod(norm(0, 1).cdf((X[i] - mu * X[i - 1]) / np.sqrt(vi * X[i - 1])))
    return first_multiplier * second_multiplier